# Introduction

Hi everyone!

This is a comprehensive report of a beginner level Bioiformatics with Python course on [Drug Discovery with Using Machine Learning and Data Analysis.](https://www.youtube.com/watch?v=jBlTQjcKuaY) The course was found on [freeCodeCamp](https://www.freecodecamp.org/news/python-for-bioinformatics-use-machine-learning-and-data-analysis-for-drug-discovery/), where you can find an abundance of free resources on data analysis and machine learning. I was very lucky to find one with applications in Bioinformatics. It is addressed to beginners, and prior knowledge of Biology or Bioinformatics is not required.

This guide I hope to be able to add more details on each step of the Data Analysis process, thus making this even more beginner friendly, if you are like me and just jumping into Data Analysis and Machine Learning. Let's dive in!

# Step 1: Acquiring the data

For this project we will be using the The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/), which is a database that contains curated bioactivity data of more than 2 million compounds.

In order to access these data, we need to install the web service package that the ChEMBL Database kindly provides.

In order to do so, we use the following command:

In [ ]:
! pip install chembl_webresource_client

## Understanding the ChEMPL Database

If you visit The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) website, you will come upon an interface which you can use to search for and filter various entries. For example, you can input the name of a disease in the search box (like Alzheimer's), and you will receive a plethora of results. If you filter the results to show only "Targets", you will get the *Target Proteins (or organisms)* associated to the disease.

**Target Proteins** are those that are addressed and controlled by **biologically active compounds**, meaning drugs. These are the proteins on which the drug will induce a *modulatory* activity, which can be activating or inhibiting it essentially.



## Importing the client

To be able to use the client now that we installed it, we need to import it into our python code. If you don't have pandas already installed in your environment, now is the time to do it! We are going to need this library for sure.

In [ ]:
! pip install pandas

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## Search for target protein

Just as we used the website of the ChEMBL database before, we can perform the same actions through the client we installed and imported at the start of this step. Convenient *huh*? In this example we can search for any virus or disease to find the targets and other useful data.

So in that case we could search for "coronavirus" and choose a target from the results, as shown below:

In [ ]:
# Target search for coronavirus

target = new_client.target
target_query = target.search('coronavirus')

# using pandas to turn this into a handy dataframe
targets = pd.DataFrame.from_dict(target_query)
targets

However we are going to cheat it a bit, knowing the name of the protein we are looking for, so we will search for it by name.

In [ ]:
# Target search for Acetylcholinesterase

target = new_client.target
target_query = target.search('acetylcholinesterase')

# using pandas to turn this into a handy dataframe
targets = pd.DataFrame.from_dict(target_query)
targets

## Select and retrieve bioactivity data for one single-protein target: Acetylcholinesterase  (1st entry)

In this step we use a single-protein target for which we will retrieve bioactivity data. In our case we choose the Human Acetylcholinesterase protein. To proceed we need to retrieve and store the `target_chembl_id` which is the unique identifier for this protein, so we don't have to go around calling it "single-protein target, Acetylcholinesterase of the Homo sapiens organism", as if it's royalty from medieval times.

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

Here we select only a specific type of activity, which bares the code "IC50". The reason we filter by `standard_type` is that it ensures that the bioactivity units will be uniform, so we will not have to deal with such incosistencies. 

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

## Export to CSV

We will save this bioactivity data into a csv, to be used later as input.

In [ ]:
df.to_csv('acetylcholinesterase_01_bioactivity_data_raw.csv', index=False)

# Step 2: Pre-processing our data

Now that we have retrieved our data, into a dataframe and also a csv, we can start processing them to get our own unique and cohesive DataFrame to work on.

## Handling Missing Data
The first step in the cleaning process, is getting rid of entries that have empty `standard_value` or `canonical_smiles`.

In [ ]:
df = pd.read_csv('acetylcholinesterase_01_bioactivity_data_raw.csv')
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

We see how we dropped from `8832` entries to `7547`. Now we will check if there are duplicates for `canonical_smiles`.

We call the `unique()` function to check this. If the output of unique values is smaller than the number of rows, then we will know that we have duplicate values.

In [ ]:
len(df2.canonical_smiles.unique())

Evidently the unique values are less than `7547`, so we will go ahead and drop the duplicate values.

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

### **Labeling compounds as either being active, inactive or intermediate**
 We will label the data based on their ``standard value`` which is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [ ]:
bioactivity_class = []
for i in df2_nr.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

## Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame

As we see above, it is quite hard to check all 46 columns of the table, so we will need to combine the columns we are studying, into one DataFrame.

### The molecule_chembl_id
Remember how we mentioned **Target Proteins**, and how they are the ones controlled and affected by an active compound? Well the list of molecules that we are about to iterate are those compounds. Compounds and molecules are therefore the same thing, and in our case are responsible for activating or inhibiting the target. For the sake of minimum redundancy, we will only keep one entry of each compound in our dataframe. The ``molecule_chembl_id`` is a unique identifier for these compounds.

We select it among the `canonical_smiles` and `standard_value` (IC50), to finally concatinate it with the `bioactivity_class` list from above, where we labeled them based on their IC50 value.

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]

pd.concat([df3,pd.Series(bioactivity_class)],axis=1)

Small note: we had to convert the `bioactivity_class` into a Series or else it could not be concatinated with the DataFrame.

In [ ]:
df3.to_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

In [ ]:
df4 = pd.read_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

# Step 3: Exploratory Data Analysis

Exploratory Data Analysis refers to all the investigation we perform on a dataset, whether that be spotting anomalies, testing a hypothesis, discovering patterns, or anything of the sort. In the scope of this step of the project, we will calculate descriptors. Descriptors are mathematical function that map datasets to a fixed-size vector. And because it took me reading that sentence 3 times to understand it, descriptors are calculated so you can cluster/classify/visualise your data more easily through the output of the calculation.

## Installing conda and rdkit

I have provided an additional guide to installation for a different system than the one provided originally in the course.
**Important Note** : changes to the $PATH variable done in Jupyter cells don't persist, so it is better to add conda to your path through the terminal directly. To do so type this *after* installing miniconda3 (so after the bash command):

----
```
echo 'export PATH="$HOME/miniconda3/bin:$PATH"' >> ~/.zshrc
source ~/.zshrc
```
----
Change `zshrc` with `bashrc` if that's what you're using.


In [ ]:
# for Linux with Python 3.7
# ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
# ! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
# ! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
# ! conda install -c rdkit rdkit -y
# import sys
# sys.path.append('/usr/local/lib/python3.7/site-packages/')

# for MacOS with Python 3.11.3 (I use zsh, modify PATH related command if you use bash)

! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.sh
! bash Miniconda3-latest-MacOSX-x86_64.sh -b -p $HOME/miniconda3
# add conda to your path!
! conda install -c rdkit rdkit